Misspellings of curse words?  

When building vocab, consider punctuation. Tokenize it first? Right now it just splits on whitespace. Moses probably has something for this

In [23]:
from time import time

import numpy as np
from scipy.sparse import csr_matrix, vstack

np.random.seed(1234) # help reproducibility

In [8]:
# y == 0 if not offensive
# y == 1 if offensive
with open('data/OLIDv1.0/olid-training-v1.0.tsv') as f:
    raw = [x.strip().split('\t') for x in f.readlines()[1:]]
    x_raw = [r[1] for r in raw]
    y = np.array([0 if r[2] == 'NOT' else 1 for r in raw])

In [9]:
# Build vocab
id2word = {}
word2id = {}
i = 0 # index of unique word
for tweet in x_raw:
    for word in tweet.split():
        if word not in word2id:
            word2id[word] = i
            id2word[i] = word
            i += 1

# Build bag of words
data = []
rows = []
cols = []
for i in range(len(x_raw)):
    counts = {}
    for word in x[i].split():
        if word2id[word] not in counts:
            counts[word2id[word]] = 1
        else:
            counts[word2id[word]] += 1
            
    for word_id, freq in counts.items():
        data.append(freq)
        rows.append(i)
        cols.append(word_id)
        
    # bias
    data.append(1)
    rows.append(i)
    cols.append(len(word2id))
        
x = csr_matrix((data, (rows, cols)))

In [10]:
# Shuffle x and y together
state = np.random.get_state()
i = np.arange(x.shape[0])
np.random.shuffle(i)
x =  x[i, :] # shuffling a sparse matrix is weird
np.random.set_state(state)
np.random.shuffle(y)

In [11]:
def sigmoid(x):
    s = np.zeros(x.shape)
    s[x > 0] = 1. / (1. + np.exp(-x[x > 0])) # avoid overflow
    s[x <= 0] = np.exp(x[x <= 0]) / (np.exp(x[x <= 0]) + 1) # avoid underflow
    return s

def cost(w, x, y):
    """ The cost function for logistic regression """
    h = sigmoid(x @ w)
    cost = np.average(-y * np.log(h) - (1 - y) * np.log(1 - h))
    return cost

def gradient_update(w, x, y):
    """ The gradient update for logistic regression"""
    h = sigmoid(x @ w)
    g = (h - y) @ x
    g = g / x.shape[0] 
    return g

def gradient_descent(w, x, y, alpha, iterations, print_iterations):
    """ Batch gradient descent algorithm """
    alpha *= x.shape[0]
    for i in range(iterations):
        if i % print_iterations == 0:
            print('%d) cost: %f' % (i, cost(w, x, y)))
        w -= alpha * gradient_update(w, x, y)       
    return w

def predict(w, x):
    """ Predict whether the label is 0 or 1 using learned logistic regression parameters """
    h = x @ w
    probabilities = sigmoid(h)
    predicted = 1 * (h > 0) # converts truth values to 1 or 0
    return probabilities, 1 * predicted

In [24]:
initial_w = np.random.rand(x.shape[1]) - 0.5
alpha = 0.00005
iterations = 10000
print_iterations = 1000
start = time()
w = gradient_descent(initial_w, x, y, alpha, iterations, print_iterations)
print('took %.2fs' % (time() - start))

0) cost: 0.944414
1000) cost: 0.507481
2000) cost: 0.434892
3000) cost: 0.389039
4000) cost: 0.355499
5000) cost: 0.329135
6000) cost: 0.307490
7000) cost: 0.289180
8000) cost: 0.273353
9000) cost: 0.259472


In [26]:
prob, labels = predict(w, x)
acc = labels[np.where(labels == y)].size / float(y.size) * 100
print('accuracy: %.4f%%' % acc)

accuracy: 92.5982%


In [26]:
# k-fold cross validation
k = 5
chunk_size = x.shape[0] // k
alpha = 0.00005
iterations = 10000
print_iterations = 1000
total_acc = 0
for i in range(k):
    start = time()
    initial_w = np.random.rand(x.shape[1]) - 0.5
    x_train = vstack((x[:i * chunk_size], x[(i + 1) * chunk_size:]))
    y_train = np.concatenate((y[:i * chunk_size], y[(i + 1) * chunk_size:]))
    x_val = x[i * chunk_size:(i + 1) * chunk_size]
    y_val = y[i * chunk_size:(i + 1) * chunk_size]
    w = gradient_descent(initial_w.copy(), x_train, y_train, alpha, iterations, print_iterations)
    _, labels = predict(w, x_train)
    train_acc = labels[np.where(labels == y_train)].size / float(y_train.size) * 100
    print('train acc: %.4f%%' % train_acc)
    _, labels = predict(w, x_val)
    val_acc = labels[np.where(labels == y_val)].size / float(y_val.size) * 100
    total_acc += val_acc
    print('val acc: %.4f%%' % val_acc)
    print('took %.2fs' % (time() - start))
print('avg acc: %.4f%%' % (total_acc / k))

0) cost: 0.898328
1000) cost: 0.513092
2000) cost: 0.438068
3000) cost: 0.390530
4000) cost: 0.355756
5000) cost: 0.328487
6000) cost: 0.306193
7000) cost: 0.287442
8000) cost: 0.271343
9000) cost: 0.257299
train acc: 92.6832%
val acc: 72.9985%
took 24.80s
0) cost: 0.824515
1000) cost: 0.514268
2000) cost: 0.437343
3000) cost: 0.388827
4000) cost: 0.353596
5000) cost: 0.326172
6000) cost: 0.303820
7000) cost: 0.285059
8000) cost: 0.268970
9000) cost: 0.254944
train acc: 92.9097%
val acc: 72.0166%
took 24.57s
0) cost: 0.892208
1000) cost: 0.514598
2000) cost: 0.438955
3000) cost: 0.390718
4000) cost: 0.355583
5000) cost: 0.328360
6000) cost: 0.306113
7000) cost: 0.287403
8000) cost: 0.271333
9000) cost: 0.257309
train acc: 92.8531%
val acc: 74.6601%
took 25.49s
0) cost: 0.972615
1000) cost: 0.510333
2000) cost: 0.435265
3000) cost: 0.388068
4000) cost: 0.353770
5000) cost: 0.326836
6000) cost: 0.304770
7000) cost: 0.286169
8000) cost: 0.270160
9000) cost: 0.256164
train acc: 92.7209%
va